## This code analyzes binary trais with missing value that are not labeld as "identity"

In [1]:
import sys
!{sys.executable} -m pip install openpyxl --no-index

Ignoring pip: markers 'python_version < "3"' don't match your environment
Looking in links: /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/avx2, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/generic, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic
Processing /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic/openpyxl-3.1.1+computecanada-py2.py3-none-any.whl
Processing /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic/et_xmlfile-1.1.0+computecanada-py3-none-any.whl


In [45]:
import csv
import pandas as pd
import numpy as np

#binary traits WITH missing values that are not labeled "identity"
rows_with_NA_ex_identity = []

with open('/home/mikekaz/scratch/CLSA_Mike/Regenie/Binary_Pheno (1).csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if int(row['Missing_values']) > 0 and row['Label'] != 'identity':
            rows_with_NA_ex_identity.append(row['Column'])

In [46]:
print(f"In total, we have {len(rows_with_NA_ex_identity)} binary traits that are not labeled 'identity' and have missing values")

In total, we have 520 binary traits that are not labeled 'identity' and have missing values


### The ones with missing value less than 1000 NAs (~4% CLSA pop)

In [47]:
#binary traits WITH missing values that are not labeled "identity"
rows_with_NA_ex_identity_low_missing = []

with open('/home/mikekaz/scratch/CLSA_Mike/Regenie/Binary_Pheno (1).csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if row['Column'] in rows_with_NA_ex_identity  and int(row['Missing_values']) <= 1000:
            rows_with_NA_ex_identity_low_missing.append(row['Column'])
            
print(len(rows_with_NA_ex_identity_low_missing))
print(rows_with_NA_ex_identity_low_missing)

36
['STP_DOTRECR_COM', 'STP_WORRECR_COM', 'STP_COLRECR_COM', 'FAS_RECR_F_COM', 'FAS_RECR_A_COM', 'FAS_RECR_S_COM', 'SPA_SOAC_RNP_COM', 'SPA_SOAC_HY_COM', 'SPA_SOAC_HIC_COM', 'SPA_SOAC_HOC_COM', 'SPA_SOAC_DT_COM', 'SPA_SOAC_INT_COM', 'SPA_SOAC_VOT_COM', 'SPA_SOAC_NONE_COM', 'SPA_SOAC_DK_NA_COM', 'SPA_SOAC_REFUSED_COM', 'WLK_RES_STOP_COM', 'WLK_SMPTS_CHT_COM', 'WLK_SMPTS_DIZ_COM', 'WLK_SMPTS_PAIN_COM', 'WLK_SMPTS_NONE_COM', 'TBI_TYP_VH_COM', 'TBI_TYP_FL_COM', 'TBI_TYP_SPT_COM', 'TBI_TYP_NONE_COM', 'TBI_TYP_OT_COM', 'TBI_TYP_DK_NA_COM', 'TBI_TYP_REFUSED_COM', 'VA_GLASSES_COM', 'VA_CNTLENS_COM', 'VA_DONE_R_COM', 'VA_DONE_L_COM', 'VA_DONE_R_PO_COM', 'VA_DONE_L_PO_COM', 'VA_DONE_BOTH_COM', 'BLD_DRAW_COM']


In [48]:
#make a phenotype file for these traits
additional_column_to_load = 'ADM_GWAS3_COM'
columns_to_load = rows_with_NA_ex_identity_low_missing + [additional_column_to_load]
columns_to_load = list(set(columns_to_load))

file_path = "~/projects/rrg-vmooser/shared/CLSA/Original/2109005_McGill_DTaliun_Baseline/2109005_McGill_DTaliun_Baseline_CoPv7.csv"
df = pd.read_csv(file_path, usecols=columns_to_load, low_memory=False)

In [49]:
df = df.dropna(subset=['ADM_GWAS3_COM'])

# Remove the decimal part and the trailing '.0' from the ADM_GWAS3_COM column
df['ADM_GWAS3_COM'] = df['ADM_GWAS3_COM'].apply(lambda x: str(int(x)) if pd.notna(x) else '')

print(f'The filtered DataFrame has {df.shape[0]} rows.')


The filtered DataFrame has 26622 rows.


In [50]:
# Read the list of IDs of interest from the text file
with open("/home/mikekaz/scratch/CLSA_Mike/Regenie/EUR_ID_Regenie.txt", "r") as f:
    id_list = f.read().splitlines()

#subset Europeans
df = df[df["ADM_GWAS3_COM"].isin(id_list)]
print(f'The filtered DataFrame has {df.shape[0]} rows.')
print(df["ADM_GWAS3_COM"].nunique())

The filtered DataFrame has 25254 rows.
25254


In [51]:
#PHENOTYPE

Phe_temp = pd.DataFrame({
    "FID": df["ADM_GWAS3_COM"].values,
    "IID": df["ADM_GWAS3_COM"].values,
})
# Save the dataframe to a CSV file
Phe_temp.to_csv("Phe_temp.csv", index=False)
# Read Phe_Regenie.csv and df.csv
Phe_temp = pd.read_csv('Phe_temp.csv')
print(Phe_temp.shape)

(25254, 2)


In [52]:
# Convert the datatype of the 'ADM_GWAS3_COM' column in df to int64
df['ADM_GWAS3_COM'] = df['ADM_GWAS3_COM'].astype('int64')


# Merge the datasets on the common column (ADM_GWAS3_COM in df1, and IID in df2)
merged_df = Phe_temp.merge(df, left_on='IID', right_on='ADM_GWAS3_COM')
merged_df.shape

(25254, 39)

In [53]:
merged_df.head()

,FID,IID,STP_DOTRECR_COM,STP_WORRECR_COM,STP_COLRECR_COM,FAS_RECR_F_COM,FAS_RECR_A_COM,FAS_RECR_S_COM,SPA_SOAC_RNP_COM,SPA_SOAC_HY_COM,...,TBI_TYP_REFUSED_COM,VA_GLASSES_COM,VA_CNTLENS_COM,VA_DONE_R_COM,VA_DONE_L_COM,VA_DONE_R_PO_COM,VA_DONE_L_PO_COM,VA_DONE_BOTH_COM,BLD_DRAW_COM,ADM_GWAS3_COM
0,24986,24986,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,24986
1,7565,7565,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,...,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,7565
2,16899,16899,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,16899
3,11060,11060,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,...,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,11060
4,19437,19437,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,19437


In [54]:
merged_df=merged_df.drop('ADM_GWAS3_COM', axis=1)

def float_to_int(x):
    if pd.isna(x):
        return x
    return int(x)


merged_df = merged_df.applymap(float_to_int)

# Modify the values in the FID and IID columns
merged_df['FID'] = merged_df['FID'].astype(str) + '_' + merged_df['FID'].astype(str)
merged_df['IID'] = merged_df['IID'].astype(str) + '_' + merged_df['IID'].astype(str)

In [55]:
#excluding traits that have sd=0
print(merged_df.shape)

# Calculate the standard deviation of each column
std_devs = merged_df.std()

# Get a list of columns where standard deviation is zero
cols_to_drop = std_devs[std_devs == 0].index

# Drop these columns from the dataframe
merged_df = merged_df.drop(cols_to_drop, axis=1)

print(merged_df.shape)

(25254, 38)
(25254, 38)


/tmp/ipykernel_703371/1978106484.py:5: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  std_devs = merged_df.std()


In [56]:
# write the dataframe back to a CSV file with NaN values replacing any missing values
merged_df.to_csv('rows_with_NA_ex_identity_low_missing.csv',sep=' ', index=False, na_rep='NA', float_format='%.0f')


merged_df.head()

,FID,IID,STP_DOTRECR_COM,STP_WORRECR_COM,STP_COLRECR_COM,FAS_RECR_F_COM,FAS_RECR_A_COM,FAS_RECR_S_COM,SPA_SOAC_RNP_COM,SPA_SOAC_HY_COM,...,TBI_TYP_DK_NA_COM,TBI_TYP_REFUSED_COM,VA_GLASSES_COM,VA_CNTLENS_COM,VA_DONE_R_COM,VA_DONE_L_COM,VA_DONE_R_PO_COM,VA_DONE_L_PO_COM,VA_DONE_BOTH_COM,BLD_DRAW_COM
0,24986_24986,24986_24986,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
1,7565_7565,7565_7565,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,16899_16899,16899_16899,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
3,11060_11060,11060_11060,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
4,19437_19437,19437_19437,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0


### The ones with missing value more than 1000 NAs (~4% CLSA pop)

In [57]:
rows_with_NA_ex_identity_high_missing = [item for item in rows_with_NA_ex_identity if item not in rows_with_NA_ex_identity_low_missing]
len(rows_with_NA_ex_identity_high_missing)

484

In [58]:
merged_df.head()

,FID,IID,STP_DOTRECR_COM,STP_WORRECR_COM,STP_COLRECR_COM,FAS_RECR_F_COM,FAS_RECR_A_COM,FAS_RECR_S_COM,SPA_SOAC_RNP_COM,SPA_SOAC_HY_COM,...,TBI_TYP_DK_NA_COM,TBI_TYP_REFUSED_COM,VA_GLASSES_COM,VA_CNTLENS_COM,VA_DONE_R_COM,VA_DONE_L_COM,VA_DONE_R_PO_COM,VA_DONE_L_PO_COM,VA_DONE_BOTH_COM,BLD_DRAW_COM
0,24986_24986,24986_24986,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
1,7565_7565,7565_7565,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,16899_16899,16899_16899,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
3,11060_11060,11060_11060,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
4,19437_19437,19437_19437,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
